In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import pickle

In [2]:
path = '/Users/alexcappadona/Desktop/JARVIS/aaa/'
fname = 'Asset_and_Accrual_Detail_20220304151810_204982643.xls'
loc = path+fname
print(loc)

/Users/alexcappadona/Desktop/JARVIS/aaa/Asset_and_Accrual_Detail_20220304151810_204982643.xls


In [3]:
ass = pd.read_excel(loc)

In [4]:
ass.head()

,Reporting Account Number,Reporting Account Name,Source Account Number,Source Account Name,As Of Date,Security Description 1,Security Description 2,Asset Code,Asset Category,Sector Name,...,Underlying Security QUICK,Underlying Security Loan ID,Underlying Security MANAGER,Accounting Status,Primary GSP Account,Extended GSP Account Number,Option Type,Market Type,Fair Value Hierarchy Level,Fair Value Hierarchy Override
0,WV9F10000002,WVU SMIF,WV9F10000002,WVU SMIF,2022-03-03,AMN HEALTHCARE SERVICES INC,NaN,E039,EQUITY,COMMERCIAL SERVICES,...,NaN,NaN,NaN,PRELIMINARY,652280,652280,NaN,NaN,1.0,NO
1,WV9F10000002,WVU SMIF,WV9F10000002,WVU SMIF,2022-03-03,CASH,NaN,NaN,CASH & CASH EQUIVALENTS,CASH,...,NaN,NaN,NaN,PRELIMINARY,652280,652280,NaN,NaN,NaN,NaN
2,WV9F10000002,WVU SMIF,WV9F10000002,WVU SMIF,2022-03-03,AMAZON.COM INC,NaN,E079,EQUITY,INTERNET,...,NaN,NaN,NaN,PRELIMINARY,652280,652280,NaN,NaN,1.0,NO
3,WV9F10000002,WVU SMIF,WV9F10000002,WVU SMIF,2022-03-03,AMERICAN CAMPUS COMMUNITIES IN,NaN,E3,EQUITY,EQUITY REIT (REAL ESTATE INVESTMENT TRUST),...,NaN,NaN,NaN,PRELIMINARY,652280,652280,NaN,NaN,1.0,NO
4,WV9F10000002,WVU SMIF,WV9F10000002,WVU SMIF,2022-03-03,APPLE INC,NaN,E091,EQUITY,COMPUTERS,...,NaN,NaN,NaN,PRELIMINARY,652280,652280,NaN,NaN,1.0,NO


## get_assets

In [5]:
# takes DataFrame of portfolio data
# creating new DataFrame with security, base cost, and market value for all assets
def get_assets(ass):
    asset_name = pd.DataFrame(ass['Security Description 1'])
    base_price = pd.DataFrame(ass['Base Cost'])
    market_price = pd.DataFrame(ass['Base Market Value'])
    frames = [asset_name,base_price,market_price]
    result = pd.concat(frames, axis=1)
    return result

## drop_cash

In [6]:
# takes a dataframe of portfolio assets
# returns a dataframe without cash and receivable for investments sold
def drop_cash(ass_frame):
    sec_frame = ass_frame.drop([0,1])
    return sec_frame

## port_aum

In [7]:
# takes dataframe of portfolio assets
# returns DataFrame with the current date and assets under management
def port_aum(ass_frame):
    base_mv = ass_frame['Base Market Value'].tolist()
    aum = sum(base_mv)
    
    today = datetime.today().strftime('%Y-%m-%d')
    entry = {"AUM": pd.Series(aum, index=[today])}
    aum = pd.DataFrame(entry)
    
    return aum

In [40]:
# made up entry for testing
fake_entry = pd.DataFrame({"AUM": pd.Series(540000.69, index=['2022-03-04'])})
fake_entry

,AUM
2022-03-04,540000.69


## record_aum

In [8]:
# takes the port_aum data frame
# records it in the historic ledger of aum via pickle

def record_aum(new_entry):
    rfile = open("aum.pickle","rb")
    aum_data = pickle.load(rfile)
    
    aum_record = pd.concat([aum_data,new_entry])
    wfile = open("aum.pickle","wb")
    pickle.dump(aum_record,wfile)
    
    return aum_record

## get_aum_record

In [13]:
# takes no arugments
# checks what is saved in aum.pickle

def get_aum_record():
    rfile = open("aum.pickle","rb")
    aum_data = pickle.load(rfile)
    return aum_data

In [14]:
get_aum_record()

,AUM
2022-03-03,538082.19
2022-03-04,536282.05


## date_drop

In [10]:
# takes a date, file
# drops the date in file
def drop_date(date,file):
    rfile = open(file,"rb")
    aum_data = pickle.load(rfile)
    
    new_aum_data = aum_data.drop(labels=[date])
    
    wfile = open("aum.pickle","wb")
    pickle.dump(new_aum_data,wfile)
    return

In [27]:
aum_frame = get_aum_record()

In [28]:
aum_frame

,AUM
2022-03-03,538082.19
2022-03-04,536282.05


## aum_delta

In [68]:
# takes aum_frame
# returns one day price change in aum

def aum_delta(aum_frame):
    aum_lst = aum_frame['AUM'].tolist()
    periods = len(aum_lst)
    current_pd = int(periods - 1)
    last_pd = int(periods - 2)
    delta = round((aum_lst[current_pd] - aum_lst[last_pd]),2)
    return delta

## aum_delta_per

In [86]:
# takes aum_frame
# returns one day percent change in aum
def aum_delta_per(aum_frame):
    aum_lst = aum_frame['AUM'].tolist()
    periods = len(aum_lst)
    current_pd = int(periods - 1)
    last_pd = int(periods - 2)
    delta_per = (aum_lst[current_pd]/aum_lst[last_pd])-1
    return delta_per

In [87]:
per_change = aum_delta_per(aum_frame)

In [88]:
per_change = round(per_change*100,2)

In [89]:
print(str(per_change)+'%')

-0.33%
